In [1]:
from xml.dom.minidom import parse
import xml.dom.minidom
import json
import numpy as np
import scipy.io as scio
import os
import math
from os import path
import pandas as pd

	


def get_node_value(element,name):
	if len(element.getElementsByTagName(name)[0].childNodes)>0:
		return element.getElementsByTagName(name)[0].childNodes[0].nodeValue
	else:
		return None



DOMTree = xml.dom.minidom.parse('283_68124copy.xml')
collection= DOMTree.documentElement

utterances=collection.getElementsByTagName('UTTERANCE')

videos=['2011-12-01_0043-cam1-for-ss3',
       '2011-12-08_0045-cam1-for-ss3',
       '2011-12-08_0046-cam1-for-ss3',
       '2012-01-27_0050-cam1-for-ss3',
       '2012-01-27_0051-cam1-for-ss3',
       '2012-01-27_0052-cam1-for-ss3',
       '2012-01-27_0053-cam1-for-ss3',
       '2012-01-27_0055-cam1-for-ss3',
       '2012-01-27_0056-cam1-for-ss3',
       '2012-02-09_0057-cam1-for-ss3',
       '2012-02-09_0058-cam1-for-ss3',
       '2012-02-09_0059-cam1-for-ss3',
       '2012-02-09_0061-cam1-for-ss3',
       '2012-02-14_0063-cam1-for-ss3',
       '2012-02-14_0065-cam1-for-ss3',
       '2013-06-27_CB_0107-cam1-for-ss3',
       '2013-06-27_CB_0110-cam1-for-ss3',
       '2013-06-27_CB_0111-cam1-for-ss3',
       '2013-06-27_CB_0112-cam1-for-ss3']

l={}
for video in videos:
    l[video]=[]



ut={}
s=[]
count =0
for utterance in utterances:
    count+=1
    ut={}
    uid = utterance.getAttribute('ID')
    stf= int(math.ceil(eval(utterance.getAttribute('START_FRAME'))/1001))
    edf= int(math.ceil(eval(utterance.getAttribute('END_FRAME'))/1001))
    ut['utterance_id']=uid
    ut['start_frame']=stf
    ut['end_frame']=edf
    ut['sign']={}

    a = utterance.getElementsByTagName('MANUALS')
    video = utterance.getElementsByTagName('MEDIA-FILES')
    video = video[0].getElementsByTagName('MEDIA-FILE')[0].getAttribute('NAME')
    video= str(video)[:-4]
    # if not os.path.exists(video):
    # 	os.mkdir(video)


    b=a[0].getElementsByTagName('SIGN')
    for c in b:
        ID = c.getAttribute('ID')
        LABEL=get_node_value(c,'LABEL')
        SIGN_TYPE = get_node_value(c,'SIGN_TYPE')
        TWO_HANDED = get_node_value(c,'TWO_HANDED')
        PASSIVE_BASE_ARM =  get_node_value(c,'PASSIVE_BASE_ARM')
        MARKED_HANDS = get_node_value(c,'MARKED_HANDS')
        START_END_HS = get_node_value(c,'START_END_HS')
        START_END_LEFT_HS =get_node_value(c,'START_END_LEFT_HS')
        START_END_RIGHT_HS = get_node_value(c,'START_END_RIGHT_HS')
        D_START_HS = get_node_value(c,'D_START_HS')
        ND_START_HS = get_node_value(c,'ND_START_HS')
        D_END_HS = get_node_value(c,'D_END_HS')
        ND_END_HS = get_node_value(c,'ND_END_HS')
        TWOHANDED_HANDSHAPES = get_node_value(c,'TWOHANDED_HANDSHAPES')
        DOMINANT_HAND_START_FRAME=eval(c.getElementsByTagName('DOMINANT_HAND')[0].getAttribute('START_FRAME'))/1001
        DOMINANT_HAND_END_FRAME=eval(c.getElementsByTagName('DOMINANT_HAND')[0].getAttribute('END_FRAME'))/1001
        ut['sign'][ID]=[ID,LABEL,SIGN_TYPE,TWO_HANDED,PASSIVE_BASE_ARM,MARKED_HANDS,START_END_HS,START_END_LEFT_HS,START_END_RIGHT_HS,D_START_HS,ND_START_HS,D_END_HS ,ND_END_HS,TWOHANDED_HANDSHAPES,DOMINANT_HAND_START_FRAME,DOMINANT_HAND_END_FRAME]
    l[video].append(ut)

In [27]:
label=set()
label_type=set()
max_time = 0
min_time = 100
count_time=0
count=0
count2=0
for key in l.keys():
    for ut in l[key]:
        for value in ut['sign'].values():
            start= value[-2]
            end = value[-1]
            if end-start>10:
                count +=1
            count2+=1
            max_time = max(max_time,end-start)
            min_time = min(min_time,end-start) 
            label_type.add(value[2])
print(count/count2)

0.16876015959804935


In [28]:
label_type

{'Classifiers',
 'Fingerspelled Signs',
 'Gestures',
 'Lexical Signs',
 'Loan Signs'}

In [20]:

for v in videos:
    for ut in l[v]:
        uid=ut['utterance_id']
        s = ut['start_frame']
        e = ut['end_frame']
        sign =[]
        for value in ut['sign'].values():
            sign.append((value[0],value[1],value[2],value[-2],value[-1]))
        df = pd.DataFrame(sign,columns  =['ID','LABEL','SIGN_TYPE','DOMINANT_HAND_START_FRAME','DOMINANT_HAND_END_FRAME']) 
        file= './sign/label_type/'+str(v)+'_'+str(uid)+'_'+str(s)+'to'+str(e)+'.csv'
        df.to_csv(file)



In [72]:
s = {}

for v in videos:
    time = 0
    for i in l[v]:
        ss={}
        for sign in i['sign']:
            ss[sign[1]]=sign[-1]-sign[-2]
        for key in ss.keys():
            #time+=ss[key]
            time+=2
    s[v]  = time

        

In [73]:
s2={}

for v in videos:
    time = 0
    for i in l[v]:
        time +=  i['end_frame']-i['start_frame']
    s2[v]  = time

In [74]:
s2

{'2011-12-01_0043-cam1-for-ss3': 3599,
 '2011-12-08_0045-cam1-for-ss3': 3629,
 '2011-12-08_0046-cam1-for-ss3': 2915,
 '2012-01-27_0050-cam1-for-ss3': 2131,
 '2012-01-27_0051-cam1-for-ss3': 3422,
 '2012-01-27_0052-cam1-for-ss3': 1856,
 '2012-01-27_0053-cam1-for-ss3': 2015,
 '2012-01-27_0055-cam1-for-ss3': 2780,
 '2012-01-27_0056-cam1-for-ss3': 1088,
 '2012-02-09_0057-cam1-for-ss3': 2227,
 '2012-02-09_0058-cam1-for-ss3': 3908,
 '2012-02-09_0059-cam1-for-ss3': 2051,
 '2012-02-09_0061-cam1-for-ss3': 3758,
 '2012-02-14_0063-cam1-for-ss3': 5444,
 '2012-02-14_0065-cam1-for-ss3': 967,
 '2013-06-27_CB_0107-cam1-for-ss3': 12098,
 '2013-06-27_CB_0110-cam1-for-ss3': 6415,
 '2013-06-27_CB_0111-cam1-for-ss3': 2712,
 '2013-06-27_CB_0112-cam1-for-ss3': 12833}

In [75]:
s3 = {}
for key in s2.keys():
    s3[key] = s[key]/s2[key]*100

In [78]:
a = l['2013-06-27_CB_0112-cam1-for-ss3']

In [81]:
for u in a:
    for sign in u['sign']:
        print(u['utterance_id'])
        print(sign)

8394379
['1950', 'FATHER', 'Lexical Signs', 'F', 'F', 'F', 'SAME', None, None, '5', None, '5', None, None, 399.0, 406.0]
8394379
['1950', 'FATHER', 'Lexical Signs', 'F', 'F', 'F', 'SAME', None, None, '5', None, '5', None, None, 399.0, 406.0]
8394379
['1951', 'ARRIVE', 'Lexical Signs', 'T', 'F', 'F', 'SAME', None, None, 'B-L', 'B-L', 'B-L', 'B-L', 'SAME', 414.0, 421.0]
8394379
['1951', 'ARRIVE', 'Lexical Signs', 'T', 'F', 'F', 'SAME', None, None, 'B-L', 'B-L', 'B-L', 'B-L', 'SAME', 414.0, 421.0]
8394379
['1952', 'HOME', 'Lexical Signs', 'F', 'F', 'F', 'SAME', None, None, 'flat-O', None, 'flat-O', None, None, 431.0, 440.0]
8394379
['1952', 'HOME', 'Lexical Signs', 'F', 'F', 'F', 'SAME', None, None, 'flat-O', None, 'flat-O', None, None, 431.0, 440.0]
8394379
['1953', 'fs-LATE', 'Fingerspelled Signs', 'F', 'F', 'F', 'DIFFERENT', None, None, 'L', None, 'X', None, None, 445.0, 458.0]
8394379
['1953', 'fs-LATE', 'Fingerspelled Signs', 'F', 'F', 'F', 'DIFFERENT', None, None, 'L', None, 'X', No